In [1]:
from ase.build import fcc100, add_adsorbate, bulk
from ase.io import write
import ase
from catkit.gen.adsorption import get_adsorption_sites
import catkit
import numpy as np


# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
vacuum = 15.0
slab = fcc100('Cu', size=(4,4,4), a=Cu_alat, vacuum=vacuum)
write('Cu_slab.cif', slab)

# try build slab using ASE -- no option for full set (not sym reduced) set of adsorption sites
free_slab = slab.copy()
# possible sites are ‘ontop’, ‘hollow’, ‘hollow’.
add_adsorbate(free_slab, 'Ag', 0, 'hollow')
write('hollow_slab.cif', free_slab)

# free_slab.get_surface_atoms()

# free_slab_gratoms = catkit.gratoms.Gratoms(free_slab)
# get_adsorption_sites(free_slab_gratoms)

# write('abs_slab.cif', abs_slab)
# slab.center(vacuum=10.0, axis=2)

In [2]:
def get_random_idx(connectivity, type=None):
    """Get random site index
    """
    connectivities = {
        'top': 1,
        'bridge': 2,
        'hollow': 4
    } # defaults to hollow

    # top should have connectivity 1, bridge should be 2 and hollow more like 4

    if type:
        # TODO: implement preferred connecitivity
        pass
    
    rand_idx = random.randrange(len(connectivity))

    return rand_idx

In [3]:
# build surface using catkit instead
a1 = bulk('Cu', 'fcc', a=Cu_alat)
catkit_slab = catkit.build.surface(a1, size=(4,4,4), miller=(1,0,0), termination=0, fixed=0, vacuum=15, orthogonal=False)
write('gratoms.cif', catkit_slab)
# written out .cif file looks good

#SlabGenerator-> layer_type='angs', layers=10...

# get ALL the adsorption sites
# top should have connectivity 1, bridge should be 2 and hollow more like 4
coords, connectivity, sym_idx = get_adsorption_sites(catkit_slab, symmetry_reduced=False)
print(f"In pristine slab, there are a total of {len(connectivity)} sites")

# keeping track of state
import numpy as np
state = np.zeros(len(coords), dtype=int)

# adsorb to a random site
import random
random.seed(7)
rand_idx = get_random_idx(connectivity)
# rand_idx = len(coords)-1
rand_site = coords[rand_idx]
print(f"idx is {rand_idx} at {rand_site} with connectivity {connectivity[rand_idx]}")

Ag = catkit.gratoms.Gratoms('Cu')
# tag the atom to be adsorbed with its to-be index (last position on slab)    
adsorbate_idx = len(catkit_slab)
# tags don't seem to be useful rn
# Ag.set_tags(adsorbate_idx)
state[rand_idx] = adsorbate_idx
catkit_slab_build = catkit.gen.adsorption.Builder(catkit_slab)
catkit_slab_w_adsorbate = catkit_slab_build.add_adsorbate(Ag, bonds=[0], index=rand_idx, auto_construct=False, symmetric=False)
write(f'gratoms_adsorbed.cif', catkit_slab_w_adsorbate)

# get adsorption sites for catkit_slab_w_adsorbate
_, new_connectivity, _ = get_adsorption_sites(catkit_slab_w_adsorbate, symmetry_reduced=False)
print(f"In adsorbed slab, there are a total of {len(new_connectivity)} sites") # which is the same as the number in pristine slab

# try to adsorb to the same site using another atom -- worked, still the same atom
Na = catkit.gratoms.Gratoms('Na')
# Na.set_tags(rand_idx)
catkit_slab_ads_build = catkit.gen.adsorption.Builder(catkit_slab_w_adsorbate)
catkit_slab_again_adsorbate = catkit_slab_ads_build.add_adsorbate(Na, bonds=[0], index=rand_idx, auto_construct=False, symmetric=False)
write(f'gratoms_again_adsorbed.cif', catkit_slab_again_adsorbate)

/Users/xiaochendu/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/Users/xiaochendu/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r2_topology = np.array(self.r2_topology)


In pristine slab, there are a total of 64 sites
idx is 41 at [ 6.3899472   7.66793665 20.42205   ] with connectivity 2
In adsorbed slab, there are a total of 64 sites


In [9]:
# adsorb a Cu on a different site 
rand_idx_2 = get_random_idx(connectivity)
rand_site_2 = coords[rand_idx_2]
print(f"idx is {rand_idx_2} at {rand_site_2} with connectivity {connectivity[rand_idx_2]}")

# Cu = catkit.gratoms.Gratoms(a1)
Cu = catkit.gratoms.Gratoms('Cu')
# tag the atom to be adsorbed with its to-be index (last position on slab)    
adsorbate_idx_2 = len(catkit_slab_w_adsorbate)
# Au.set_tags(adsorbate_idx_2)
state[rand_idx_2] = adsorbate_idx_2
catkit_slab_build_2 = catkit.gen.adsorption.Builder(catkit_slab_w_adsorbate)
catkit_slab_w_2_adsorbates = catkit_slab_build_2.add_adsorbate(Cu, bonds=[0], index=rand_idx_2, auto_construct=False, symmetric=False)
write(f'gratoms_adsorbed_2.cif', catkit_slab_w_2_adsorbates)

# desorb Ag
# checking no double adsorption
# slab_tags = catkit_slab_w_adsorbate.get_tags()
# assert len(np.argwhere(slab_tags==adsorbate_idx)) <= 1, "more than 1 site found"
# assert len(np.argwhere(slab_tags==adsorbate_idx)) == 1, "no sites found"
# np.argwhere(slab_tags==adsorbate_idx)[0,0]
assert len(np.argwhere(state==adsorbate_idx)) <= 1, "more than 1 site found"
assert len(np.argwhere(state==adsorbate_idx)) == 1, "no sites found"

# remove adsorbate
gatoms_for_del = catkit_slab_w_2_adsorbates.copy()
del gatoms_for_del[adsorbate_idx]

# assert slab after deletion is the same as slab before adsorption
# print(gatoms_for_del == catkit_slab)
write(f"gratoms_2_delete_1.cif", gatoms_for_del)

idx is 50 at [ 1.27798944  6.3899472  20.42205   ] with connectivity 4


/Users/xiaochendu/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/Users/xiaochendu/miniconda3/envs/mlenv/lib/python3.9/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r2_topology = np.array(self.r2_topology)


In [5]:
assert len(np.argwhere(state==adsorbate_idx_2)) <= 1, "more than 1 site found"
assert len(np.argwhere(state==adsorbate_idx_2)) == 1, "no sites found"

# try remove Au, failed because the Ag before it was already removed
gatoms_for_del = gatoms_for_del.copy()
print(f"type of {adsorbate_idx_2} is {type(state[rand_idx_2])}")
try:
    del gatoms_for_del[state[rand_idx_2]]
except IndexError as e:
    print(f"error message: {e}")

# now actually delete
gatoms_for_del[adsorbate_idx_2-1]
write(f"gratoms_2_delete_2.cif", gatoms_for_del)

type of 65 is <class 'numpy.int64'>
error message: index 65 is out of bounds for axis 0 with size 65


In [6]:
state = np.where(state>0,state-1,0)

In [7]:
state

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 64,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [8]:
# from catkit.gen.surface import SlabGenerator
# from ase.visualize import view

# # Make a test slab
# atoms = bulk('Pd', 'fcc', cubic=True)
# atoms[3].symbol = 'Cu'

# gen = SlabGenerator(
#     atoms,
#     miller_index=(2, 1, 1),
#     layers=9,
#     fixed=5,
#     vacuum=4)
# free_slab.info